**Linear Regression**<br>
By: Laksh Patel<br><br>

**Prerequisites:**<br>
Facility in Linear Algebra & Multi-Variable Calculus<br><br>

**Intuition:**<br>
We begin with the approximate system $A\vec x\approx \vec b$, where $A_{m\times n}$ contains $n$ feature columns and $m$ observations, $\vec x$ is the coefficient vector, and $\vec b$ is the vector of observed outputs. When $A\vec x=\vec b$ is exactly solvable, $\vec b$ lies in the column space $C(A)$.<br><br>

In general $\vec b\notin C(A)$, so the best attainable prediction is the orthogonal projection of $\vec b$ onto $C(A)$, denoted $\vec b'=\mathrm{proj}_{C(A)}(\vec b)$. The goal of linear regression is to find the $\vec x$ for which $A\vec x$ equals this projection.<br><br>

To obtain such an $\vec x$, define the least-squares loss $L(\vec x)=\|A\vec x-\vec b\|^2$. Expanding, $\|A\vec x-\vec b\|^2=(A\vec x-\vec b)^T(A\vec x-\vec b)$, and differentiating gives $\nabla L(\vec x)=2A^T(A\vec x-\vec b)$. Setting this to zero produces the normal equations $A^TA\,\vec x=A^T\vec b$.<br><br>

The solution to these equations is given by the pseudoinverse formula $\vec x=A^+\vec b$, which yields the unique minimum-norm vector among all least-squares minimizers and is valid regardless of whether $A$ is full-rank or rank-deficient.<br><br>

In all cases, $A\vec x$ is the point in $C(A)$ closest to $\vec b$ in Euclidean distance.

**Application:**<br>
Now that we've defined the mathematical ideology, let's think algorithmically.

1. Take the data matrix $A$ and observation vector $\vec b$.  
2. Form the normal-equation components: compute $A^TA$ and $A^T\vec b$.  
3. Compute the pseudoinverse $A^+$ (typically via SVD).  
4. Solve for the coefficients using $\vec x=A^+\vec b$.  
5. Compute the prediction $\hat b=A\vec x$.  
6. Compute the residual $r=\vec b-\hat b$ and its norm $\|r\|$ if needed.  
7. Interpret $\hat b$ as the projection of $\vec b$ onto $C(A)$.


In [ ]:
#Let's define some matrix operations

def transpose(M):
    return [list(row) for row in zip(*M)] #Swaps rows for columns

def matmul(A, B):
    # matrix * matrix or matrix * vector
    if isinstance(B[0], list):
        # A (m×n) * B (n×p)
        return [[sum(a*b for a, b in zip(A_row, B_col))
                 for B_col in zip(*B)]
                for A_row in A]
    else:
        # A (m×n) * b (n)
        return [sum(a*b for a, b in zip(A_row, B)) for A_row in A]

def identity(n):
    return [[1 if i==j else 0 for j in range(n)] for i in range(n)]

def invert_matrix(M):
    # Gauss–Jordan elimination
    n = len(M)
    A = [row[:] for row in M]
    I = identity(n)

    for i in range(n):
        # Pivot
        pivot = A[i][i]
        if pivot == 0:
            raise ValueError("Matrix not invertible.")
        inv_pivot = 1 / pivot

        # Normalize pivot row
        for j in range(n):
            A[i][j] *= inv_pivot
            I[i][j] *= inv_pivot

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r][i]
                for c in range(n):
                    A[r][c] -= factor * A[i][c]
                    I[r][c] -= factor * I[i][c]

    return I

In [ ]:


# -----------------------------
# Linear Regression
# -----------------------------

def linear_regression(A, b):
    A_T = transpose(A)                  # A^T
    ATA = matmul(A_T, A)                # A^T A
    ATb = matmul(A_T, b)                # A^T b  (vector)

    # Solve (A^T A)x = A^T b
    ATA_inv = invert_matrix(ATA)
    x = matmul(ATA_inv, ATb)

    # Prediction
    b_hat = matmul(A, x)

    # Residual
    residual = [bi - hi for bi, hi in zip(b, b_hat)]

    return x, b_hat, residual


# -----------------------------
# Example
# -----------------------------

A = [
    [1, 1],
    [1, 2],
    [1, 3]
]

b = [1, 2, 2]

x, pred, r = linear_regression(A, b)

print("Coefficients x =", x)
print("Prediction A x =", pred)
print("Residual =", r)

Coefficients x = [0.6666666666666679, 0.5]
Prediction A x = [1.1666666666666679, 1.6666666666666679, 2.166666666666668]
Residual = [-0.16666666666666785, 0.33333333333333215, -0.16666666666666785]
